In [1]:
import requests
import datetime
import inspect
import json
from typing import (
    TypedDict, 
    List, Dict, Literal, 
    Callable, Optional, Any, 
    get_type_hints
)
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(base_url="https://openai.vocareum.com/v1")

In [3]:
class Memory:
  def __init__(self):
    self._messages: List[Dict[str, str]] = []

  def add_message(self, 
                  role: Literal["user", "system", "assistant"], 
                  content: str,
                  tool_calls: dict=dict(),
                  tool_call_id=None)->None:

    message = {
      "role": role,
      "content": content,
      "tool_calls": tool_calls
    }
    
    if role == "tool":
      message = {
        "role": role,
        "content": content,
        "tool_call_id": tool_call_id
      }
    
    self._messages.append(message)
  
  
  def get_messages(self) -> List[Dict[str, str]]:
    return self._messages
  
  
  def last_message(self) -> Dict[str, str]:
    if self._messages:
      return self._messages[-1]


  def reset(self) -> None:
    self._messages = []


In [4]:
def chat_with_tools(user_question: str=None,
                    memory:Memory=None,
                    model: str="gpt-4o-mini",
                    temperature=0.0,
                    tools=None) -> str:
  
  messages = [{"role": "user", "content": user_question}]
  if memory:
    if user_question:
      memory.add_message(role="user", content=user_question)
    
    messages = memory.get_messages()
  
  response = client.chat.completions.create(
    model=model,
    temperature=temperature,
    messages=messages,
    tools=tools,
  )
  
  ai_message = str(response.choices[0].message.content)
  tool_calls = response.choices[0].message.tool_calls
  
  if memory:
    memory.add_message(role="assistant", content=ai_message, tool_calls=tool_calls)
  
  return ai_message


In [5]:
class Tool:
  def __init__(self, func: Callable):
    self.func = func
    self.name = func.__name__
    self.description = func.__doc__
    self.argument_types_map = get_type_hints(func)
    self.signature = inspect.signature(func)
    self.arguments = [
      {
        "name": key,
        "type": self._infer_json_schema_type(value),
        "required": param.default == inspect.Parameter.empty
      }
      
      for key, value in self.argument_types_map.items()
      if (param := self.signature.parameters.get(key))
    ]

  def _infer_json_schema_type(self, arg_type: Any) -> str:
    if arg_type == bool:
      return "boolean"
    elif arg_type == int:
      return "integer"
    elif arg_type == float:
      return "number"
    elif arg_type == str:
      return "string"
    elif arg_type == list:
      return "array"
    elif arg_type == dict:
      return "object"
    elif arg_type is None:
      return "null"
    elif arg_type == datetime.datetime or arg_type == datetime.date:
      return "string"
    else:
      return "string"
    
      
  def dict(self):
    return {
      "type": "function",
      "function": {
        "name": self.name,
        "description": self.description,
        "parallel_tool_calls": False,
        "parameters": {
          "type": "object",
          "properties": {
            argument["name"]: {
              "type": argument["type"],
            }
            for argument in self.arguments
          },
          "required": [
            argument["name"]
            for argument in self.arguments
            if argument["required"]
          ],
          "additionalProperties": False
        },
        "strict": True
      }
    }

  def __call__(self, *args, **kwargs):
    return self.func(*args, **kwargs)
  

In [6]:
def power(base:float, exponent:float):
  """Exponentatiation: base to the power of exponent"""
  
  return base ** exponent

In [7]:
power_tool = Tool(power)

In [8]:
power_tool.dict()

{'type': 'function',
 'function': {'name': 'power',
  'description': 'Exponentatiation: base to the power of exponent',
  'parallel_tool_calls': False,
  'parameters': {'type': 'object',
   'properties': {'base': {'type': 'number'}, 'exponent': {'type': 'number'}},
   'required': ['base', 'exponent'],
   'additionalProperties': False},
  'strict': True}}

In [9]:
power(2, 3)

8

In [ ]:
class Agent:
  """A tool-calling AI agent"""
  
  def __init__(
    self,
    name: str = "Agent",
    role: str = "Personal Assistant",
    instructions: str = "Help users with any question.",
    model: str = "gpt-4o-mini",
    temperature: float = 0.0,
    tools: List[Tool] = [],
    ):
    self.name = name
    self.role = role
    self.instructions = instructions
    self.model = model
    self.temperature = temperature
    self.memory = Memory()
    self.memory.add_message(
      role="system",
      content=f"You're an AI Agent, your rolse is {self.role}, "
              f"and you need to {self.instructions}"
    )
    self.client = client
    self.tools = tools
    self.tool_map = {t.name: t for t in tools}
    self.openai_tools = [t.dict() for t in self.tools] if self.tools else None
  
  def invoke(self, 
             user_message: str, 
             self_reflection: bool = False, 
             max_iter: int = 1,
             verbose: bool = False) -> str:
    
    self.memory.add_message(
      role="user",
      content=user_message
    )
    
    if verbose:
      self._log_last_message()
    
    max_iter = max_iter if max_iter >= 1 else 1
    max_iter = max_iter if max_iter <= 3 else 3
    max_iter = max_iter if self_reflection else 0.5
    loops = 2 * max_iter
    
    for i in range(loops):
      ai_message = self._get_completion(
        messages = self.memory.get_messages()
      )
      
      self.memory.add_message(
        role = "assistant",
        content = ai_message.content,
      )
      
      if verbose:
        self._log_last_message()
      
      if i < loops -1:
        self.memory.add_message(
          role = "user",
          content = self.critique_prompt
        )
        
        if verbose:
          self._log_last_message()
        
        ai_message = self._get_completion(
          messages = self.memory.get_messages()
        )

  def _get_completion(self, messages: List[Dict]) -> ChatCompletionMessage:
    response = self.client.chat.completions.create(
      model=self.model,
      temperature=self.temperature,
      messages=messages
    )
    
    return response.choices[0].message
  
  def _log_last_message(self):
    print(f"### {self.memory.last_message()['role']} message ### \n".upper())
    print(f"### {self.memory.last_message()['content']} \n")
    print("\n__________________________________________________________________")
